In [11]:
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import sqlite3
import pymongo
from geopy.distance import geodesic
import geocoder
from geopy.geocoders import Nominatim
from twilio.rest import Client
import datetime

class Responses:
    incident_mapping = {
        "fire": {
            "collections": ["firestations", "hospitals", "policestations"],
            "service_labels": ["Fire Station", "Hospital", "Police Station"],
        },
        "fight": {
            "collections": ["policestations"],
            "service_labels": ["Police Station"],
        },
        "accident": {
            "collections": ["hospitals", "policestations"],
            "service_labels": ["Hospital", "Police Station"],
        },
    }
    
    def __init__(self):
        self.client = pymongo.MongoClient("mongodb://localhost:27017/")
        self.db = self.client["SERVICES"]
        self.camera_latitude, self.camera_longitude = 28.55678, 77.28322
        self.final_services = []

    def get_location(self, lat, lon):
        geoLoc = Nominatim(user_agent="GetLoc")
        location = geoLoc.reverse((lat, lon))
        return location.address

    def calculate_nearest_services(self, incident_type):
        if incident_type in self.incident_mapping:
            incident_data = self.incident_mapping[incident_type]
            service_labels = self.incident_mapping[incident_type]["service_labels"]

            nearest_services = {}
            for i, collection_name in enumerate(incident_data["collections"]):
                nearest_services[collection_name] = []

                for service in self.db[collection_name].find():
                    service_location = (service["LATITUDE"], service["LONGITUDE"])
                    distance = geodesic((self.camera_latitude, self.camera_longitude), service_location).kilometers

                    nearest_services[collection_name].append({
                        "name": service["NAME"],
                        "service": incident_data["service_labels"][i],
                        "distance": distance,
                    })

            for collection_name in nearest_services:
                nearest_services[collection_name].sort(key=lambda x: x["distance"])

            for collection_name in nearest_services:
                if nearest_services[collection_name]:
                    nearest_service = nearest_services[collection_name][0]
                    self.final_services.append(nearest_service)

        else:
            print(f"Invalid incident type: {incident_type}")


    def get_service_location(self):
        latitude = []
        longitude = []
        for service in self.final_services:
            service_name = service['name']
            service_type = service['service']

            if service_type == "Fire Station":
                collection_name = "firestations"
            elif service_type == "Hospital":
                collection_name = "hospitals"
            elif service_type == "Police Station":
                collection_name = "policestations"

            service_data = self.db[collection_name].find_one({"NAME": service_name})

            if service_data:
                latitude_number = service_data.get('LATITUDE', '')
                longitude_number = service_data.get('LONGITUDE', '')

                latitude.append(latitude_number)
                longitude.append(longitude_number)

        return latitude, longitude

    def generate_directions_urls(self, latitude, longitude):
        directions_urls = []

        for i in range(len(latitude)):
            start_latitude = latitude[i]
            start_longitude = longitude[i]

            url = f"https://www.google.com/maps/dir/{start_latitude},{start_longitude}/{self.camera_latitude},{self.camera_longitude}"
 
            directions_urls.append(url)

        return directions_urls

    def send_sms(self, to_phone_number, message):
        account_sid = 'ACa7f320a488f1812d0b10de2be6be2217'
        auth_token = 'cba2eaf51227cb598e0a3aea479a5b86'
        from_phone_number = '+18142994366'

        client = Client(account_sid, auth_token)
        client.messages.create(
            body=message,
            from_=from_phone_number,
            to=to_phone_number
        )
     

    def notify_via_sms(self, incident_type, incident_location, current_datetime, service_name, phone_number, url):
        message = f'''
        Dear {service_name},

        An incident has been detected at the following location:

        Type of Incident: {incident_type}
        Latitude: {self.camera_latitude}
        Longitude: {self.camera_longitude}
        Address of Accident: {incident_location}
        Google Map Link (Route): {url}

        Please respond promptly to this location.

        Additional Information:
        Date and Time: {current_datetime}

        Sincerely,
        Your Emergency Notification System
        '''

#         self.send_sms(phone_number, message)
        print(f"SMS sent to {service_name}")

    def send_email(self, subject, message, recipient_email):
        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'aznakhkydseu@gmail.com'
        sender_password = 'azal otlv kvkl uzjp'

        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject

        msg.attach(MIMEText(message, 'plain'))

        try:
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
            print('Email sent successfully.')
            server.quit()
        except Exception as e:
            print('Email sending failed.')
            print(str(e))
     
    def notify_via_email(self, incident_type, accident_location, current_datetime, service_name, email_address, url):
        subject = 'Emergency Notification'

        message = f'''
        Dear {service_name},

        An incident has been detected at the following location:

        Type of Incident: {incident_type}
        Latitude: {self.camera_latitude}
        Longitude: {self.camera_longitude}
        Address of Incident: {accident_location}

        Google Map Link (Route): {url}

        Please respond promptly to this location.

        Additional Information:
        Date and Time: {current_datetime}

        Sincerely,
        Your Emergency Notification System
        '''

        self.send_email(subject, message, email_address)
        print(f"Email sent to {service_name}")

    def get_collection_name(self, service_type):
        if service_type == "Fire Station":
            return "firestations"
        elif service_type == "Hospital":
            return "hospitals"
        elif service_type == "Police Station":
            return "policestations"
        else:
            raise ValueError(f"Invalid service type: {service_type}")

    def get_service_data(self, service):
        return self.db[self.get_collection_name(service['service'])].find_one({"NAME": service['name']})
    
    def get_incident_details(self, camera_latitude, camera_longitude, incident_type):
        self.final_services = []  # Reset final_services for each query

        # Calculate nearest services based on the provided incident type
        self.calculate_nearest_services(incident_type)

        # Get incident location details
        incident_location = self.get_location(camera_latitude, camera_longitude)

        # Generate URLs for directions
        latitude, longitude = self.get_service_location()
        directions_urls = self.generate_directions_urls(latitude, longitude)

        confirmation_lines_email = []
        confirmation_lines_sms = []
        current_datetime = datetime.datetime.now()

        result = []

        for service, url in zip(self.final_services, directions_urls):
            service_name = service['name']
            service_data = self.get_service_data(service)
            phone_number = service_data.get('PHN NO.', '')
            email_address = service_data.get('EMAIL ID', '')

            urls = f" {url} "
            confirmation_email = f"Email sent to {service_name}"
            confirmation_sms = f"SMS sent to {service_name}"
            email_address = f" {email_address} "
            service_details = {
                'Name': service_name,
                'Service Type': service['service'],
                'Distance': service['distance'],
                'Phone Number': phone_number,
                'Email Address': email_address,
                'Direction URL': urls,
                'Email confirmation': confirmation_email,
                'SMS confirmation': confirmation_sms
            }

            result.append(service_details)

            
            self.notify_via_email(incident_type, incident_location, current_datetime, service_name, email_address, urls)
            self.notify_via_sms(incident_type, incident_location, current_datetime, service_name, phone_number, urls)

        return {
            'TYPE OF INCIDENT': incident_type,
            'DATE and TIME OF INCIDENT': current_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3],
            'LATITUDE OF INCIDENT': camera_latitude,
            'LONGITUDE OF INCIDENT': camera_longitude,
            'ADDRESS OF INCIDENT': incident_location,
            'SERVICE DETAILS': result
        }


In [2]:
from Detection.utils.main_utils import Responses
# Usage example:
response_obj = Responses()
camera_latitude = 28.56234651631763
camera_longitude = 77.28039429363223
incident_type = "fire"
result = response_obj.get_incident_details(camera_latitude, camera_longitude, incident_type)
# 28.56234651631763, 77.28039429363223
# 28.55678, 77.28322

KeyboardInterrupt: 

In [13]:
result

{'TYPE OF INCIDENT': 'fire',
 'DATE and TIME OF INCIDENT': '2023-12-02 20:23:16.589',
 'LATITUDE OF INCIDENT': 28.56234651631763,
 'LONGITUDE OF INCIDENT': 77.28039429363223,
 'ADDRESS OF INCIDENT': 'Jamia Millia Islamia, Maulana Mohamed Ali Jauhar Marg, Okhla, Defence Colony Tehsil, South East Delhi District, Delhi, 110025, India',
 'SERVICE DETAILS': [{'Name': 'FIRE STATION NEHRU PLACE',
   'Service Type': 'Fire Station',
   'Distance': 2.8353098346080765,
   'Phone Number': '+919821540868',
   'Email Address': ' an3895@dseu.ac.in ',
   'Direction URL': ' https://www.google.com/maps/dir/28.557168117337305,77.25424768869934/28.55678,77.28322 ',
   'Email confirmation': 'Email sent to FIRE STATION NEHRU PLACE',
   'SMS confirmation': 'SMS sent to FIRE STATION NEHRU PLACE'},
  {'Name': 'IQBAL FAIZI',
   'Service Type': 'Hospital',
   'Distance': 0.6048329557311849,
   'Phone Number': '+919821540868',
   'Email Address': ' an3895@dseu.ac.in ',
   'Direction URL': ' https://www.google.com

In [7]:
import time
start = time.time()
for i in range(100000000):
    continue
end=time.time()
print(end-start)

7.0527379512786865
